# Encrypted Inference Example

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Datasets
%install '.package(path: "$cwd/ppmlNB_tensor")' ppmlNB_tensor

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Datasets
	.package(path: "/root/swift-ppml/secure_computation/ppmlNB_tensor")
		ppmlNB_tensor
With SwiftPM flags: []
Working in: /tmp/tmpfqiglkn3/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
import ppmlNB_tensor
import TensorFlow

## Public Training

In [ ]:
import Datasets

let epochCount = 12
let batchSize = 128

let dataset = MNIST()

var classifier = Sequential {
    Flatten<Float>()
    Dense<Float>(inputSize: 784, outputSize: 10)
}

let optimizer = SGD(for: classifier, learningRate: 0.1)

print("Beginning training...")

struct Statistics {
    var correctGuessCount: Int = 0
    var totalGuessCount: Int = 0
    var totalLoss: Float = 0
    var batches: Int = 0
}

let testBatches = dataset.testDataset.batched(batchSize)

// The training loop.
for epoch in 1...epochCount {
    var trainStats = Statistics()
    var testStats = Statistics()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(epoch))

    Context.local.learningPhase = .training
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        // Compute the gradient with respect to the model.
        let 𝛁model = TensorFlow.gradient(at: classifier) { classifier -> Tensor<Float> in
            let ŷ = classifier(images)
            let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
            trainStats.correctGuessCount += Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
            trainStats.totalGuessCount += batchSize
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.totalLoss += loss.scalarized()
            trainStats.batches += 1
            return loss
        }
        // Update the model's differentiable variables along the gradient vector.
        optimizer.update(&classifier, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in testBatches {
        let (labels, images) = (batch.label, batch.data)
        // Compute loss on test set
        let ŷ = classifier(images)
        let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
        testStats.correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
        testStats.totalGuessCount += batchSize
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.totalLoss += loss.scalarized()
        testStats.batches += 1
    }

    let trainAccuracy = Float(trainStats.correctGuessCount) / Float(trainStats.totalGuessCount)
    let testAccuracy = Float(testStats.correctGuessCount) / Float(testStats.totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(trainStats.totalLoss / Float(trainStats.batches)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(trainAccuracy)), \
        Test Loss: \(testStats.totalLoss / Float(testStats.batches)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(testAccuracy))
        """)
}

Loading resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
2020-02-05 05:53:18.499367: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
2020-02-05 05:53:18.556751: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-02-05 05:53:18.582334: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2020-02-05 05:53:18.584070: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14734f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-02-05 05:53:18.584118: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-02-05 05:53:18.588581: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t1

In [ ]:
// Extract weights
let w1 = classifier.layer2.weight
let b1 = classifier.layer2.bias

In [ ]:
func get_input(_ nb_input: Int) -> (Tensor<Float>, Tensor<Int32>){
    let dataset = MNIST()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, 
        randomSeed: Int64(1)).batched(nb_input)
    var train_iter = trainingShuffled.self.makeIterator()
    var image = train_iter.next().unsafelyUnwrapped
    return (image.data.reshaped(to: [nb_input, 784]), image.label)
    
}

In [ ]:
let (input, label) = get_input(3)

## Encrypted Inference

In [ ]:
func private_prediction(input: Tensor<Float>) -> PrivateTensor{
    let input_private = PrivateTensor.from_values(values: input)
    let w1_private = PrivateTensor.from_values(values: w1)
    let b1_private = PrivateTensor.from_values(values: b1)
    let private_pred = add(x: matmul(x: input_private, y: w1_private), y: b1_private)
    return private_pred
}

In [ ]:
for i in 0...2 {
    var private_output = private_prediction(input: input[i].reshaped(to: [1,784]))
    var decode_prediction = private_output.reveal().decode()
    var private_class = decode_prediction.argmax(squeezingAxis: 1)[0]
    var exp_class = label[i]
    print("Private prediction result: \(private_class). Expected prediction: \(exp_class)")
}